In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path
from datetime import datetime

In [2]:
def process_excel(input_path, output_folder):
    try:
        # 读取数据
        df1 = pd.read_excel(input_path, sheet_name='Sheet1')
        df2 = pd.read_excel(input_path, sheet_name='Sheet2')

        # 第一步：核对总数量
        total1 = df1['数量'].sum()
        total2 = df2['数量'].sum()
        
        # 第二步：详细型号核对
        group1 = df1.groupby('规格型号')['数量'].sum().reset_index()
        group2 = df2.groupby('规格型号')['数量'].sum().reset_index()
        merged = pd.merge(group1, group2, on='规格型号', how='outer', 
                        suffixes=('_发票', '_订单')).fillna(0)

        # 定义三种异常情况
        condition_dict = {
            '无订货': merged['数量_订单'] == 0,
            '未开票': merged['数量_发票'] == 0,
            '数量不符': (merged['数量_发票'] > 0) & 
                      (merged['数量_订单'] > 0) & 
                      (merged['数量_发票'] != merged['数量_订单'])
        }

        # 收集所有异常型号
        error_data = []
        for status, cond in condition_dict.items():
            temp = merged[cond].copy()
            temp['状态'] = status
            error_data.append(temp[['规格型号', '状态']])
        
        error_list = pd.concat(error_data).drop_duplicates()

        # 第三步：生成详细异常表
        sheet3_data = []
        for _, row in error_list.iterrows():
            model = row['规格型号']
            status = row['状态']
            
            # 获取两边数据
            left = df1[df1['规格型号'] == model].add_prefix('发票_').reset_index(drop=True)
            right = df2[df2['规格型号'] == model].add_prefix('订单_').reset_index(drop=True)
            
            # 对齐行数
            max_row = max(len(left), len(right))
            combined = pd.concat([
                left.reindex(range(max_row)),
                right.reindex(range(max_row))
            ], axis=1)
            
            # 添加状态列
            combined['异常类型'] = status
            sheet3_data.append(combined)

        # 合并结果
        final_sheet3 = pd.concat(sheet3_data, ignore_index=True) if sheet3_data else pd.DataFrame()
        
        # 生成输出文件名
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = Path(output_folder) / f"核对结果_{timestamp}.xlsx"
        
        # 保存结果
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # 原始数据
            df1.to_excel(writer, sheet_name='原始发票', index=False)
            df2.to_excel(writer, sheet_name='原始订单', index=False)
            
            # 异常数据（如果有）
            if not final_sheet3.empty:
                # 列排序调整
                cols = [c for c in final_sheet3.columns if c != '异常类型'] + ['异常类型']
                final_sheet3[cols].to_excel(writer, sheet_name='异常明细', index=False)
                
                # 生成统计表
                stats = error_list.groupby('状态').size().reset_index(name='数量')
                stats.to_excel(writer, sheet_name='异常统计', index=False)

        # 生成报告
        report_msg = f"总数量核对：{'一致' if total1 == total2 else '不一致'}\n"
        report_msg += "异常类型统计：\n" + "\n".join([f"{row['状态']}: {row['数量']}个" 
                                      for _, row in stats.iterrows()])

        messagebox.showinfo(
            "处理完成",
            f"{report_msg}\n\n结果文件已保存至：\n{output_path}"
        )

    except Exception as e:
        messagebox.showerror("处理错误", f"错误详情：{str(e)}")

In [3]:
# GUI界面（保持不变）
def create_gui():
    root = tk.Tk()
    root.title("数据核对系统 v2.0")
    root.geometry("650x220")

    # 文件选择组件
    def select_file():
        path = filedialog.askopenfilename(filetypes=[("Excel文件", "*.xlsx")])
        if path:
            input_entry.delete(0, tk.END)
            input_entry.insert(0, path)

    def select_folder():
        path = filedialog.askdirectory()
        if path:
            output_entry.delete(0, tk.END)
            output_entry.insert(0, path)

    # 界面布局
    tk.Label(root, text="源文件路径:").grid(row=0, column=0, padx=10, pady=10, sticky='e')
    input_entry = tk.Entry(root, width=50)
    input_entry.grid(row=0, column=1, padx=5, pady=5)
    tk.Button(root, text="选择文件", command=select_file).grid(row=0, column=2, padx=5, pady=5)

    tk.Label(root, text="输出目录:").grid(row=1, column=0, padx=10, pady=5, sticky='e')
    output_entry = tk.Entry(root, width=50)
    output_entry.grid(row=1, column=1, padx=5, pady=5)
    tk.Button(root, text="选择目录", command=select_folder).grid(row=1, column=2, padx=5, pady=5)

    # 运行按钮
    run_btn = tk.Button(root, text="开始核对", command=lambda: process_excel(input_entry.get(), output_entry.get()),
                       bg="#2196F3", fg="white", height=1, width=15)
    run_btn.grid(row=2, column=1, pady=15)

    return root

In [4]:
if __name__ == "__main__":
    gui = create_gui()
    gui.mainloop()